In [8]:
import sys
sys.path.insert(0, '../')
from qhdopt import QHD
import numpy as np

In [9]:
Q = np.array([[0.8931, 0.8673],
     [0.8673, 0.9079]])
b = np.array([0.8483, 0.8762])

In [16]:
def run_qhd(matrix, vector):
    vector = -1*vector
    model = QHD.QP(matrix, vector)
    model.dwave_setup(10, api_key_from_file='/Users/samuelkushnir/Documents/dwave_api_key.txt')
    model.optimize(verbose=0)
    return model.info["refined_minimizer"]

In [18]:
def IR_QHD(matrix, vector, IRprecision):
  ### Scaled Iterative Refinement for solving a linear system
  ### matrix: coefficient matrix of the linear system
  ### vector: right-hand side vector of the linear system
    nabla             = 1                             # Scaling factor
    rho               = 2                             # Incremental scaling
    d                 = len(matrix)                   # Dimension
    iteration         = 0
    x                 = np.zeros(d)                   # Solution
    r                 = vector                        # Residual
    con               = np.linalg.cond(matrix)             # Condition number
    res=[]
    while (np.linalg.norm(r)>IRprecision):
        # c = np.linalg.solve(matrix, nabla*r)          # Should be replaced by QHD c = QHD(matrix, nabla*r)
        c = run_qhd(matrix, vector)

        x = x + (1/nabla)*c                           # Updating solution
        r = vector - np.dot(matrix, x)                # Calculating resisdual
        nabla = min(rho*nabla,1/(np.linalg.norm(r)))  # Updating scaling factor
        res.append(np.linalg.norm(r))
        iteration=iteration+1
    return res, c

print(IR_QHD(Q, b, 1e-4))

Received Task from D-Wave:
2024-01-21 22:49:33
Backend QPU Time: 0.02525277
Overhead Time: 4.025589276737671
([1.4865979986850008e-05], array([0.17483117, 0.79807703]))
